In [1]:
import base64
import requests
import datetime
from urllib.parse import urlencode
from pprint import pprint

In [2]:
client_id = '6275b1d049e94e5ba0b8e9c54ace6902'
client_secret = 'bb7d902ac227494a898d2e04e3b2fbca'

In [3]:
# Do a lookup for a token to use it for all future requests in session

In [4]:
class SpotifyApi(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if not client_secret or not client_id:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client")
            
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {"Authorization": f"Bearer {access_token}"}
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')

    def get_track(self, _id):
        return self.get_resource(_id, resource_type='tracks')

    def get_audio_analysis(self, _id):
        return self.get_resource(_id, resource_type='audio-analysis')
    
    def base_search(self, query_params, search_type='artist'): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if not query:
            raise Exception("A query is required")
            
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
            
        if operator and operator_query:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
            
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)



In [5]:
spotify = SpotifyApi(client_id, client_secret)

In [6]:
response = spotify.search(query={"track": "Urban Twilight"}, operator='not', operator_query='grimes', search_type="track")

pprint(response)

q=track%3AUrban+Twilight+NOT+grimes&type=track
{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AUrban+Twilight+NOT+grimes&type=track&offset=0&limit=20',
            'items': [{'album': {'album_type': 'single',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5XWHODeAWqM44qXaQwGGT7'},
                                              'href': 'https://api.spotify.com/v1/artists/5XWHODeAWqM44qXaQwGGT7',
                                              'id': '5XWHODeAWqM44qXaQwGGT7',
                                              'name': 'Oeil',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:5XWHODeAWqM44qXaQwGGT7'}],
                                 'available_markets': ['AD',
                                                       'AE',
                                                       'AG',
                                      

                                             'OM',
                                             'PA',
                                             'PE',
                                             'PG',
                                             'PH',
                                             'PK',
                                             'PL',
                                             'PS',
                                             'PT',
                                             'PW',
                                             'PY',
                                             'QA',
                                             'RO',
                                             'RS',
                                             'RU',
                                             'RW',
                                             'SA',
                                             'SB',
                                             'SC',
                               

                                             'SR',
                                             'ST',
                                             'SV',
                                             'SZ',
                                             'TD',
                                             'TG',
                                             'TH',
                                             'TJ',
                                             'TL',
                                             'TN',
                                             'TO',
                                             'TR',
                                             'TT',
                                             'TV',
                                             'TW',
                                             'TZ',
                                             'UA',
                                             'UG',
                                             'US',
                               

                       'external_urls': {'spotify': 'https://open.spotify.com/track/40pMEu3UWIJZJgtRNsxDgw'},
                       'href': 'https://api.spotify.com/v1/tracks/40pMEu3UWIJZJgtRNsxDgw',
                       'id': '40pMEu3UWIJZJgtRNsxDgw',
                       'is_local': False,
                       'name': 'Urban Twilight',
                       'popularity': 0,
                       'preview_url': 'https://p.scdn.co/mp3-preview/cd384549c5b594fe7a7a01bff6832b8e28a21235?cid=6275b1d049e94e5ba0b8e9c54ace6902',
                       'track_number': 10,
                       'type': 'track',
                       'uri': 'spotify:track:40pMEu3UWIJZJgtRNsxDgw'},
                      {'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3FNnud8pybYtY4iQ3hd4r7'},
                                              'href': 'https://api.spotify.com/v1/artists/3FNnud8pybYtY4iQ3hd4r7',
     

                                              'uri': 'spotify:artist:40ORdckZc4hDymUnqBmLEA'}],
                                 'available_markets': ['AD',
                                                       'AE',
                                                       'AG',
                                                       'AL',
                                                       'AM',
                                                       'AO',
                                                       'AR',
                                                       'AT',
                                                       'AU',
                                                       'AZ',
                                                       'BA',
                                                       'BB',
                                                       'BD',
                                                       'BE',
                                                  

                                                       'AR',
                                                       'AT',
                                                       'AU',
                                                       'AZ',
                                                       'BA',
                                                       'BB',
                                                       'BD',
                                                       'BE',
                                                       'BF',
                                                       'BG',
                                                       'BH',
                                                       'BI',
                                                       'BJ',
                                                       'BN',
                                                       'BO',
                                                       'BR',
                        

                                                       'EC',
                                                       'EE',
                                                       'EG',
                                                       'ES',
                                                       'FI',
                                                       'FJ',
                                                       'FM',
                                                       'FR',
                                                       'GA',
                                                       'GB',
                                                       'GD',
                                                       'GE',
                                                       'GH',
                                                       'GM',
                                                       'GN',
                                                       'GQ',
                        

In [7]:
response = spotify.search(query={"track": "Urban Twilight"}, search_type="track")

pprint(response)

q=track%3AUrban+Twilight&type=track
{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AUrban+Twilight&type=track&offset=0&limit=20',
            'items': [{'album': {'album_type': 'single',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/053q0ukIDRgzwTr4vNSwab'},
                                              'href': 'https://api.spotify.com/v1/artists/053q0ukIDRgzwTr4vNSwab',
                                              'id': '053q0ukIDRgzwTr4vNSwab',
                                              'name': 'Grimes',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:053q0ukIDRgzwTr4vNSwab'}],
                                 'available_markets': ['AD',
                                                       'AE',
                                                       'AG',
                                                       'AL

                                                       'FR',
                                                       'GA',
                                                       'GB',
                                                       'GD',
                                                       'GE',
                                                       'GH',
                                                       'GM',
                                                       'GN',
                                                       'GQ',
                                                       'GR',
                                                       'GT',
                                                       'GW',
                                                       'GY',
                                                       'HK',
                                                       'HN',
                                                       'HR',
                        

                       'external_ids': {'isrc': 'QZK6H2123596'},
                       'external_urls': {'spotify': 'https://open.spotify.com/track/5zH4wMAAEGNt1LAVEQgCEf'},
                       'href': 'https://api.spotify.com/v1/tracks/5zH4wMAAEGNt1LAVEQgCEf',
                       'id': '5zH4wMAAEGNt1LAVEQgCEf',
                       'is_local': False,
                       'name': 'Urban Twilight',
                       'popularity': 0,
                       'preview_url': 'https://p.scdn.co/mp3-preview/e092eb62e1e213e4e31b2785a4e5336eb9a7e9aa?cid=6275b1d049e94e5ba0b8e9c54ace6902',
                       'track_number': 6,
                       'type': 'track',
                       'uri': 'spotify:track:5zH4wMAAEGNt1LAVEQgCEf'},
                      {'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kJ6fxlh1FJJH2yHtGQNKD'},
                                              'href': 'h

                                                       'LA',
                                                       'LB',
                                                       'LC',
                                                       'LI',
                                                       'LK',
                                                       'LR',
                                                       'LS',
                                                       'LT',
                                                       'LU',
                                                       'LV',
                                                       'LY',
                                                       'MA',
                                                       'MC',
                                                       'MD',
                                                       'ME',
                                                       'MG',
                        

                                                       'XK',
                                                       'BF',
                                                       'CI',
                                                       'GB',
                                                       'GH',
                                                       'GM',
                                                       'GN',
                                                       'GW',
                                                       'IE',
                                                       'IS',
                                                       'LR',
                                                       'ML',
                                                       'MR',
                                                       'PT',
                                                       'SL',
                                                       'SN',
                        

                                                       'UA',
                                                       'UG',
                                                       'US',
                                                       'UY',
                                                       'UZ',
                                                       'VC',
                                                       'VE',
                                                       'VN',
                                                       'VU',
                                                       'WS',
                                                       'XK',
                                                       'ZA',
                                                       'ZM',
                                                       'ZW'],
                                 'external_urls': {'spotify': 'https://open.spotify.com/album/1toYe0DHFXWpg4l04ahKUa'},
                         

                                                       'XK',
                                                       'ZA',
                                                       'ZM',
                                                       'ZW'],
                                 'external_urls': {'spotify': 'https://open.spotify.com/album/5CTgT0DiZowVFfG5bkPQin'},
                                 'href': 'https://api.spotify.com/v1/albums/5CTgT0DiZowVFfG5bkPQin',
                                 'id': '5CTgT0DiZowVFfG5bkPQin',
                                 'images': [{'height': 640,
                                             'url': 'https://i.scdn.co/image/ab67616d0000b273f58d7cf70745ade925f75785',
                                             'width': 640},
                                            {'height': 300,
                                             'url': 'https://i.scdn.co/image/ab67616d00001e02f58d7cf70745ade925f75785',
                                             'wid

In [8]:
spotify.get_artist("053q0ukIDRgzwTr4vNSwab")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/053q0ukIDRgzwTr4vNSwab'},
 'followers': {'href': None, 'total': 1627850},
 'genres': ['art pop',
  'canadian electropop',
  'dance pop',
  'electropop',
  'grave wave',
  'indietronica',
  'metropopolis',
  'pop'],
 'href': 'https://api.spotify.com/v1/artists/053q0ukIDRgzwTr4vNSwab',
 'id': '053q0ukIDRgzwTr4vNSwab',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb34771f759ca81a422f5f2b57',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab6761610000517434771f759ca81a422f5f2b57',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f17834771f759ca81a422f5f2b57',
   'width': 160}],
 'name': 'Grimes',
 'popularity': 73,
 'type': 'artist',
 'uri': 'spotify:artist:053q0ukIDRgzwTr4vNSwab'}

In [9]:
spotify.get_album("1Cog6aZ6NwnJiuINc0LQEU")

{'album_type': 'single',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/053q0ukIDRgzwTr4vNSwab'},
   'href': 'https://api.spotify.com/v1/artists/053q0ukIDRgzwTr4vNSwab',
   'id': '053q0ukIDRgzwTr4vNSwab',
   'name': 'Grimes',
   'type': 'artist',
   'uri': 'spotify:artist:053q0ukIDRgzwTr4vNSwab'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CD',
  'CG',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IQ',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  

In [12]:
response = spotify.search(query={"track": "sugar"}, search_type="track")

pprint(response)

q=track%3Asugar&type=track
{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3Asugar&type=track&offset=0&limit=20',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6KImCVD70vtIoJWnq6nGn3'},
                                              'href': 'https://api.spotify.com/v1/artists/6KImCVD70vtIoJWnq6nGn3',
                                              'id': '6KImCVD70vtIoJWnq6nGn3',
                                              'name': 'Harry Styles',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:6KImCVD70vtIoJWnq6nGn3'}],
                                 'available_markets': ['AD',
                                                       'AE',
                                                       'AG',
                                                       'AL',
          

                       'available_markets': ['AD',
                                             'AE',
                                             'AG',
                                             'AL',
                                             'AM',
                                             'AO',
                                             'AR',
                                             'AT',
                                             'AU',
                                             'AZ',
                                             'BA',
                                             'BB',
                                             'BD',
                                             'BE',
                                             'BF',
                                             'BG',
                                             'BH',
                                             'BI',
                                             'BJ',
                               

                                                       'UG',
                                                       'US',
                                                       'UY',
                                                       'UZ',
                                                       'VC',
                                                       'VE',
                                                       'VN',
                                                       'VU',
                                                       'WS',
                                                       'XK',
                                                       'ZA',
                                                       'ZM',
                                                       'ZW'],
                                 'external_urls': {'spotify': 'https://open.spotify.com/album/3sSfjX4fhZonjyZ10x0l0f'},
                                 'href': 'https://api.spotify.com/v1/albums/3sSfjX4fhZ

                                             'FJ',
                                             'FM',
                                             'FR',
                                             'GA',
                                             'GB',
                                             'GD',
                                             'GE',
                                             'GH',
                                             'GM',
                                             'GN',
                                             'GQ',
                                             'GR',
                                             'GT',
                                             'GW',
                                             'GY',
                                             'HK',
                                             'HN',
                                             'HR',
                                             'HT',
                               

                                             'LV',
                                             'MC',
                                             'MD',
                                             'ME',
                                             'MG',
                                             'MH',
                                             'MK',
                                             'ML',
                                             'MN',
                                             'MO',
                                             'MR',
                                             'MT',
                                             'MU',
                                             'MV',
                                             'MW',
                                             'MX',
                                             'MY',
                                             'MZ',
                                             'NA',
                               

                                             'SZ',
                                             'TD',
                                             'TG',
                                             'TH',
                                             'TJ',
                                             'TL',
                                             'TN',
                                             'TO',
                                             'TR',
                                             'TT',
                                             'TV',
                                             'TW',
                                             'TZ',
                                             'UA',
                                             'UG',
                                             'US',
                                             'UY',
                                             'VC',
                                             'VE',
                               

                       'id': '3iSws76HjaU7k49EqJVTfF',
                       'is_local': False,
                       'name': 'Sugar, Sugar',
                       'popularity': 69,
                       'preview_url': 'https://p.scdn.co/mp3-preview/00a3f87ee79567242355fd7799e73f426e9bf3f5?cid=6275b1d049e94e5ba0b8e9c54ace6902',
                       'track_number': 2,
                       'type': 'track',
                       'uri': 'spotify:track:3iSws76HjaU7k49EqJVTfF'},
                      {'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3zmfs9cQwzJl575W1ZYXeT'},
                                              'href': 'https://api.spotify.com/v1/artists/3zmfs9cQwzJl575W1ZYXeT',
                                              'id': '3zmfs9cQwzJl575W1ZYXeT',
                                              'name': 'Men I Trust',
                                              'type': 'artist

In [14]:
song_data = spotify.get_audio_analysis("6UelLqGlWMcVH1E5c4H7lY")

In [17]:
song_data.keys()

dict_keys(['meta', 'track', 'bars', 'beats', 'sections', 'segments', 'tatums'])

In [18]:
song_data['track'].keys()

dict_keys(['num_samples', 'duration', 'sample_md5', 'offset_seconds', 'window_seconds', 'analysis_sample_rate', 'analysis_channels', 'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'codestring', 'code_version', 'echoprintstring', 'echoprint_version', 'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version'])